# Rust 入門

## 基本的な型

### 数値型
- 数値型はアルファベット一文字とビットサイズから成る
    - アルファベット部:
        - `i`: 符号あり数値型
        - `u`: 符号なし数値型
        - `f`: 浮動小数点型
    - ビットサイズ部:
        - `8`: 8bit
        - `16`: 16bit
        - `32`: 32bit
        - `64`: 64bit
        - `128`: 128bit

In [2]:
fn main() {
    // 8bit 符号あり整数
    let int8bit: i8 = -128; // -128 ~ 127

    // 8bit 符号なし整数
    let uint8bit: u8 = 255; // 0 ~ 255

    // 32bit 浮動小数点
    let float32bit: f32 = 3.141592;

    println!("i8 value: {}\nu8 value: {}\nf32 value: {}", int8bit, uint8bit, float32bit);
}
main();

i8 value: -128
u8 value: 255
f32 value: 3.141592


### 文字列型
文字列型は基本的に `str` 型のみだが、実用上 `String` 型という型も使われる

- `str`:
    - 組み込みの文字列型で、文字列スライスを表す
        - スライス:
            - メモリ上に存在する文字列データのポインタ位置と長さを示すもの
    - 扱える文字列データは固定サイズ
    - 文字列そのものの変更は不可
    - C++で言うところの `const char *` に等しい
- `String`:
    - 標準ライブラリで定義されている文字列型
    - 文字列データや長さの変更が可能
    - C++で言うところの `std::string` に等しい

`str` と `String` はいずれも UTF-8 エンコーディングされた文字列データを格納している

`String` を `&str` (文字列型の参照 = 文字列データが格納されているメモリ上のポインタ位置) に変換するときは、ポインタと文字列長のコピーのみが発生し、文字列データそのもののコピーは発生しないため、メモリを圧迫することはない

一方 `&str` を `String` に変換する場合は、メモリを新規に確保して文字列データそのものをコピーするため、長い文字列の変換を行う場合のオーバーヘッドが生じる

In [13]:
fn main() {
    // 新規にメモリを確保して "Hello, World!" という文字列を格納
    let str1: String = String::from("Hello, World!");
    
    // String型 を &str型 に変換
    // => str2 (文字列の参照: メモリ上のポインタ位置) は str1 の参照に等しい
    // ※ ただし Rust のポインタはファットポインタであり、ポインタ位置以外の情報（&str の場合は文字列の長さ情報）も保持する
    //    => 実際のポインタ位置はずれる
    let str2: &str = &str1;

    // &str型 を String型 に変換
    // => 新規にメモリを確保して str2 のポインタ位置に格納されている文字列データをコピーする
    // => str3 の参照と str2 は別のポインタ位置を指す
    let str3: String = str2.to_string();
    
    // {:p}: ポインタ位置をフォーマット
    println!("str1:\n\tstring: {}\n\tpointer: {:p}", str1, &str1);
    println!("str2:\n\tstring: {}\n\tpointer: {:p}", str2, &str2);
    println!("str3:\n\tstring: {}\n\tpointer: {:p}", str3, &str3);
}
main();

str1:
	string: Hello, World!
	pointer: 0xe951aff360
str2:
	string: Hello, World!
	pointer: 0xe951aff338
str3:
	string: Hello, World!
	pointer: 0xe951aff348


### タプル型
タプルは複数の異なる型を収めることができる集合

関数から複数の値を返すときにタプルでまとめて返すことがある

タプルは内部に格納された型の全てをまとめて1つの型を構成するため、タプル内の一部の型を後から変更することは出来ない

内部の値にアクセスするときは `.0` や `.1` のように記述する

In [16]:
fn main() {
    // タプル型は (型0, 型1, ...) で宣言する
    let tuple: (i32, &str) = (1, "2");
    println!("tuple: {}, {}", tuple.0, tuple.1);
}
main();

tuple: 1, 2


### 配列型
配列は特定の型の値を連続に収めた集合

配列のサイズは固定で、コンパイル時に定まっている必要がある

内部の値にアクセスするときは `[index]` を使って指定する

配列の参照は自動的にスライスとして扱われ、`[start..end]` のような範囲指定が可能 (このとき返る値に `end` 添字の値は含まれない)

In [17]:
fn main() {
    // 配列型は [型; サイズ] で宣言する
    // * 変数の値を後から変更したい場合は mut (mutable) を宣言する 
    let mut a: [i32; 3] = [0, 1, 2]; // 3つの i32型 で構成される配列
    let b: [i32; 3] = [0; 3]; // [0, 0, 0]
    
    a[1] = b[1]; // => a: [0, 0, 2]
    a[2] = b[2]; // => a: [0, 0, 0]
    
    // {:?}: スライスをフォーマット
    println!("{:?}", &a[0..3]); // &array[0..3] => array[0], array[1], array[2]
}
main();

[0, 0]


### ユーザ定義型
Rust では、ユーザが定義可能な型は以下の2つのみ

- 構造体:
    - `struct` で宣言する
    - 複数の異なる型の値をまとめた集合型で、各データに変数名を定義することが可能
- 列挙体:
    - `enum` で宣言する
    - ユーザが定義した各識別子（列挙子）をそのまま有限集合として持つ抽象データ型
    - Rust の列挙体は、各識別子にさらにデータを付与することができる

In [24]:
fn main() {
    // Person 構造体の定義
    struct Person {
        name: String, // 文字列型変数 name
        age: u32, // unsigned 32bit integer型変数 age
    }
    
    // 構造体の値を宣言するときは `ユーザ定義型名{変数: 値, ...}` と記述する
    let person: Person = Person{
        name: String::from("John"),
        age: 16
    };
    
    // 構造体の各データへのアクセスは `.変数名` で行う
    println!("person:\n\tname: {}\n\tage: {}", person.name, person.age);
    
    
    // Event 列挙体の定義
    enum Event {
        Quit,
        KeyDown(u8),
        MouseDown {x: i32, y: i32}
    }
    
    // 列挙体は基本的に渡された識別子によって挙動を制御したい場合に使う
    fn processEvent(event: Event) {
        // match式: パターンマッチング
        // 列挙体の識別子へのアクセスは `ユーザ定義型名::識別子` で行う
        match event {
            Event::Quit => println!("Quit..."),
            Event::KeyDown(keycode) => println!("KeyDown: code={}", keycode),
            Event::MouseDown{x, y} => println!("MouseDown: x={}, y={}", x, y),
        }
    }
    
    let event1 = Event::Quit;
    let event2 = Event::KeyDown(0);
    let event3 = Event::MouseDown {x: 10, y: 20};
    
    processEvent(event1); // => Quit...
    processEvent(event2); // => KeyDown: code=0
    processEvent(event3); // => MouseDown: x=10, y=20
}
main();

person:
	name: John
	age: 16
Quit...
KeyDown: code=0
MouseDown: x=10, y=20


## よく使う標準ライブラリ型

組み込み型は、数値型、文字列型、タプル型、配列型の4つのみである

それ以外は構造体か列挙体のユーザ定義型となっている

ここでは、標準ライブラリで定義されているユーザ定義型の内、よく使う以下の型について解説する

- `Option` 列挙体
- `Result` 列挙体
- `Vec` 構造体
- `Box` 構造体

### Option型
Option型は、データが存在する場合と存在しない場合を表現する

標準ライブラリ上では以下のように定義されている (なお `pub` キーワードは外部公開することを意味する)

```rust
pub enum Option<T> {
    None,
    Some(T),
}
```

ここで `<T>` はジェネリックな型パラメータ `T` を指定可能ということを表現する

すなわち Option型 は、任意の型を内包した抽象列挙体であり、型パラメータを指定する必要があるということである

In [29]:
fn main() {
    // Option<i32>: i32型 で具象化された Option型
    // Option<i32>型 の識別子により挙動制御
    fn processI32Option(val: Option<i32>) {
        match val {
            Option::None => println!("the value is none"),
            Option::Some(v) => println!("the value is {}", v),
        }
    }
    
    let val1: Option<i32> = Option::None;
    let val2: Option<i32> = Option::Some(10);
    
    processI32Option(val1); // => the value is none
    processI32Option(val2); // => the value is 10
}
main();

the value is none
the value is 10


### Result型
Result型は、処理の結果が成功か失敗かを表現する

```rust
pub enum Result<T, E> {
    Ok(T),
    Err(E),
}
```

処理の結果が成功の場合は、任意の型 `T` のデータを持つ `Ok(T)` 識別子を使う

逆にエラーの場合は、任意の型 `E` のデータを持つ `Err(E)` 識別子を使う

例えば `Result<i32, String>` のような 具象Result型 を宣言した場合、成功したとき数値データを取得でき、失敗したとき文字列データを取得できるようになる

一般的には、処理の結果を任意の Result型 で受け取り、match式 や if let式 でパターンマッチング（処理の分岐）を行うことになる

In [33]:
fn main() {
    let result: Result<i32, String> = Ok(200);
    
    // match による分岐
    match result {
        Ok(code) => println!("code: {}", code),
        Err(err) => println!("error: {}", err),
    }
    
    // Rust は変数を再定義可能: 上書き宣言
    let result: Result<i32, String> = Ok(201);
    
    // if let による分岐
    if let Ok(code) = result {
        println!("result is OK: {}", code);
    }
    
    /* match, if let による分岐は冗長であるため、
     * Result型 には .unwrap_or メソッドが用意されている
     */
    
    // unwrap_or(v): Ok(T) だった場合にはそのまま T を返す
    println!("OK code: {}", result.unwrap_or(-1)); // Ok(200).unwrap_or(-1) => 200
    
    let result: Result<i32, String> = Err(String::from("error"));
    
    // unwrap_or(v): Err(E) だった場合には v を返す
    println!("Error code: {}", result.unwrap_or(-1)); // Err("error").unwrap_or(-1) => -1
}
main();

code: 200
result is OK: 201
OK code: 201
Error code: -1
